###Baseline Model

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
def ts_split(series, test_size=0.2):
    split = int(len(series) * (1 - test_size))
    return series[:split], series[split:]

In [ ]:
all_results = []

group_cols = ['Category', 'Region', 'Store ID', 'Product ID']

In [ ]:
#prophet multi
for (cat, reg, store, prod), grp in df.groupby(group_cols):

    ts = grp.sort_values('Date').set_index('Date')['Demand']

    if len(ts) < 40:
        continue

    # 'train' and 'test' are Demand Series, consistent with other models
    train_series, test_series = ts_split(ts)

    # ----------------------------
    # Prophet (Multivariate)
    # ----------------------------
    regressors = ['Price', 'Discount', 'Promotion', 'Competitor Pricing']

    # Create DataFrames for Prophet that include regressors and apply the same split
    grp_sorted = grp.sort_values('Date')
    split_point = int(len(grp_sorted) * (1 - 0.2)) # Assuming ts_split uses 0.2 as default test_size

    train_df_for_prophet = grp_sorted.iloc[:split_point].copy()
    test_df_for_prophet = grp_sorted.iloc[split_point:].copy()

    # Ensure all regressors exist in the DataFrame used for Prophet
    available_regs = [r for r in regressors if r in train_df_for_prophet.columns]

    if len(available_regs) > 0:

        try:
            p_multi = Prophet()

            # Add regressors
            for r in available_regs:
                p_multi.add_regressor(r)

            # Prepare training data for Prophet
            prophet_train_m = train_df_for_prophet[['Date', 'Demand'] + available_regs].rename(
                columns={'Date': 'ds', 'Demand': 'y'}
            )

            # Prepare test data for Prophet (only 'ds' and regressors for prediction)
            prophet_test_m_features = test_df_for_prophet[['Date'] + available_regs].rename(
                columns={'Date': 'ds'}
            )

            # Fit and predict
            p_multi.fit(prophet_train_m)
            p_multi_pred = p_multi.predict(prophet_test_m_features)['yhat'].values

            all_results.append({
                'Model': 'Prophet Multivariate',
                'Category': cat,
                'Region': reg,
                'Store ID': store,
                'Product ID': prod,
                'MAPE': mean_absolute_percentage_error(test_series, p_multi_pred), # Use the original test_series for evaluation
                'R2': r2_score(test_series, p_multi_pred)
            })
        except Exception as e:
            # Print a more informative error message
            print(f"INFO:prophet:Prophet Multivariate failed for Category: {cat}, Region: {reg}, Store ID: {store}, Product ID: {prod} with error: {e}")
            p_multi_pred = None
    else:
        p_multi_pred = None

###Fine-tuned Model

In [ ]:
df['Demand_log'] = np.log1p(df['Demand'])

def cap_outliers(series, q=0.99):
    cap = series.quantile(q)
    return np.where(series > cap, cap, series)

df['Demand_log'] = (
    df.groupby(['Category','Region','Store ID','Product ID'])['Demand_log']
      .transform(cap_outliers)
)

In [ ]:
for lag in [1, 7, 14]:
    df[f'lag_{lag}'] = (
        df.groupby(['Category','Region','Store ID','Product ID'])['Demand_log']
          .shift(lag)
    )

In [ ]:
df['day'] = df['Date'].dt.day
df['month'] = df['Date'].dt.month
df['dayofweek'] = df['Date'].dt.dayofweek

In [ ]:
def train_test_split_ts(df, test_ratio=0.2):
    split = int(len(df) * (1 - test_ratio))
    return df.iloc[:split], df.iloc[split:]

In [ ]:
# No longer need to initialize results as a separate list

for (cat, reg, store, prod), grp in df.groupby(
    ['Category','Region','Store ID','Product ID']
):
    # ------------------------------------
    # Prophet - Multivariate (Fine-Tuned)
    # ------------------------------------
    regressors = ['Price', 'Discount', 'Promotion', 'Competitor Pricing']

    # Ensure 'train' and 'test' are defined for the current group and have 'Date', 'Demand_log' and regressor columns
    # Split the current group's data into train and test sets, including all necessary columns
    grp_processed = grp.dropna(subset=['Demand_log'] + regressors)
    if len(grp_processed) < 40:
        continue

    # Use train_test_split_ts which operates on DataFrames
    train, test = train_test_split_ts(grp_processed)

    available_regs = [r for r in regressors if r in train.columns]

    if len(available_regs) > 0:
        try:
            # Initialize Prophet
            p_multi = Prophet(
                yearly_seasonality=False,
                weekly_seasonality=True,
                daily_seasonality=False,
                changepoint_prior_scale=0.05
            )

            # Add regressors
            for r in available_regs:
                p_multi.add_regressor(r)

            # Prepare training data
            prophet_train_m = train[['Date', 'Demand_log'] + available_regs].copy()
            prophet_train_m = prophet_train_m.rename(
                columns={'Date': 'ds', 'Demand_log': 'y'}
            )

            # Prepare test data
            prophet_test_m = test[['Date'] + available_regs].copy()
            prophet_test_m = prophet_test_m.rename(columns={'Date': 'ds'})

            # Fit model
            p_multi.fit(prophet_train_m)

            # Predict
            forecast = p_multi.predict(prophet_test_m)
            pred = np.expm1(forecast['yhat'].values)
            actual = np.expm1(test['Demand_log'].values)

            # Evaluation
            mape = mean_absolute_percentage_error(actual, pred)
            r2 = r2_score(actual, pred)

            # Store results
            all_results.append({
                'Model': 'Prophet Multivariate (Fine-Tuned)',
                'Category': cat,
                'Region': reg,
                'Store ID': store,
                'Product ID': prod,
                'MAPE': mape,
                'R2': r2
            })

        except Exception as e:
            print(f"INFO:prophet:Prophet Multivariate (Fine-Tuned) failed for Category: {cat}, Region: {reg}, Store ID: {store}, Product ID: {prod} with error: {e}")
    else:
        # Handle case where no regressors are available for Prophet Multivariate
        pass